<a href="https://colab.research.google.com/github/darisoy/EE517_Sp21/blob/master/hw3/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🐍 Setup Python environment

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 4.0MB/s 
     |████████████████████████████████| 901kB 32.6MB/s 
     |████████████████████████████████| 3.3MB 36.9MB/s 


In [23]:
import numpy as np
import pandas as pd
import math
import torch
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from tokenizers import decoders
from sklearn.metrics import classification_report

import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 📀 Load the data

In [3]:
labels = {'O' : 0,
          'B-geo-loc' : 1,
          'I-geo-loc' : 2,
          'B-product' : 3,
          'I-product' : 4,
          'B-facility' : 5,
          'I-facility' : 6,
          'B-company' : 7,
          'I-company' : 8,
          'B-person' : 9,
          'I-person' : 10,
          'B-sportsteam' : 11,
          'I-sportsteam' : 12,
          'B-musicartist' : 13,
          'I-musicartist' : 14,
          'B-movie' : 15,
          'I-movie' : 16,
          'B-tvshow' : 17,
          'I-tvshow' : 18,
          'B-other' : 19,
          'I-other' : 20,
          }
end_token = '<END>'
beg_token = '<BEG>'

In [4]:
def get_sentences(df):
    sentences = []
    labels = []
    running_sentence = ''
    runnnig_label = []
    for idx, row in df.iterrows():
        if row.word == end_token:
            if len(running_sentence[:-1]) > 0:
                sentences.append(running_sentence[:-1])
                labels.append(runnnig_label)
            running_sentence = ''
            runnnig_label = []
        else:
            running_sentence += row.word + ' '
            runnnig_label.append(row.tag)
    return sentences, labels

def get_data(type):
    data = pd.read_csv('https://raw.githubusercontent.com/aritter/twitter_nlp/master/data/annotated/wnut16/data/' + type, delimiter='\t', names=["word", "tag"], skip_blank_lines=False, quoting=3)
    data = data.fillna({'word': end_token, 'tag': 'O'})
    data.tag = data.tag.apply((lambda x: labels[x]))
    return get_sentences(data)

# 🔐 Encode the data using BERT transformer

## Load the transformer

In [5]:
transformer_name = "distilbert-base-uncased"
transformer = DistilBertModel.from_pretrained(transformer_name)
tokenizer = DistilBertTokenizer.from_pretrained(transformer_name)
tokenizer.decoder = decoders.WordPiece()

## Get dataset embeddings

In [6]:
def get_sublist_start_end(sl, l):
    results = []
    sll = len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if tokenizer.decode(l[ind:ind+sll]) ==  tokenizer.decode(sl):
            results.append([ind,ind+sll-1])
    return results

def get_embeddings(sentences):
    transformer.eval()
    transformer.to(device)
    data = []
    with torch.no_grad():
        for sentence in tqdm(sentences):
            tokens = tokenizer.encode(sentence)
            out = transformer(torch.tensor(tokens).unsqueeze(0).to(device))
            embed = []
            for i, word in enumerate(sentence.split()):
                target = word
                target_ids = tokenizer.encode(target, add_special_tokens=False)
                start_end_found = get_sublist_start_end(target_ids , tokens)
                if len(start_end_found) > 1:
                    occurrences_pos_of_cur_word = [word_i for word_i, x in enumerate(sentence.split()) if x == word]
                    cur_occ_num =  occurrences_pos_of_cur_word.index(i)
                    target_idx = start_end_found[cur_occ_num]
                else:
                    target_idx = start_end_found[0]
                embed.append(torch.mean(out[0][0][target_idx[0]:target_idx[1]+1], 0))
            data.append(torch.stack(embed))
    return data

In [7]:
train_sentences, train_tags = get_data('train')
train_embeddings = get_embeddings(train_sentences)

In [8]:
valid_sentences, valid_tags = get_data('dev')
valid_embeddings = get_embeddings(valid_sentences)

In [9]:
test_sentences, test_tags = get_data('test')
test_embeddings = get_embeddings(test_sentences)

# 🧑‍💻 Classify the embeddings using RNN

In [26]:
# Model Definition
class RNN(nn.Module):
    def __init__(self, hidden_dim, num_layers):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.GRU(768, self.hidden_dim, self.num_layers, batch_first=True) 
        self.fc = nn.Linear(self.hidden_dim, len(labels))
    
    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        out, (h) = self.rnn(x, (h.detach()))
        out = self.fc(out) 
        return out

In [31]:
def train(classifier, device, hidden_dim, num_layers, epochs, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
    classifier.train()
    # train
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0
        for i, sentence in enumerate(train_embeddings):
            tags = torch.tensor(train_tags[i])
            sentence, tags = sentence.to(device), tags.to(device)
            optimizer.zero_grad()
            outputs = classifier(sentence.unsqueeze(dim=0))
            loss = criterion(outputs.squeeze(dim=0), tags)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print('[Epoch %d]\tTrain Loss: \t\t%.3f' % (epoch+1, running_loss / len(train_embeddings)))


In [28]:
def evaluate(classifier, device, sentences, sentence_tags):
    criterion = nn.CrossEntropyLoss()
    correct = 0
    total = 0
    running_loss = 0
    truth = []
    preds = []
    classifier.eval()
    print("Evaluating ... ")
    with torch.no_grad():
        for i, sentence in enumerate(tqdm(sentences)):
            tags = torch.tensor(sentence_tags[i])
            sentence, tags = sentence.to(device), tags.to(device)

            outputs = classifier(sentence.unsqueeze(dim=0)).squeeze(dim=0)
            pred = outputs.squeeze(dim=1).argmax(dim=1)
            loss = criterion(outputs, tags)
            running_loss += loss.item()

            correct += torch.sum(tags == pred)
            total += len(tags)

            for t in sentence_tags[i]:
                truth.append(t)
            for p in pred:
                preds.append(p.item())

    loss = running_loss/len(sentences)
    accuracy = 100*correct/total

    print('Overall Accuracy: \t%.3f%% \tloss: %.3f' % (accuracy, loss))
    return preds, truth, loss, accuracy

In [33]:

# We vary these as hyperparamters
hidden_size_variations = [64, 128]
num_layers_variations = [2, 5]
num_epoch_variations = [5, 10]
learning_rate_variations = [0.01, 0.001]

combinations = len(hidden_size_variations) * len(num_layers_variations) \
                * len(num_epoch_variations) * len(learning_rate_variations)

In [34]:
loss_accuracy_comb = [[None, None, None] for i in range(combinations)]
num_comb = 0
for hidden_dim in hidden_size_variations:
    for num_layers in num_layers_variations:
        for epochs in num_epoch_variations:
            for learning_rate in learning_rate_variations:
                # train model
                classifier = RNN(hidden_dim, num_layers).to(device)
                train(classifier, device, hidden_dim, num_layers, epochs, learning_rate)
                _, _, loss, accuracy = evaluate(classifier, device, valid_embeddings, valid_tags)
                loss_accuracy_comb[num_comb][0] = float(loss)
                loss_accuracy_comb[num_comb][1] = float(accuracy)
                loss_accuracy_comb[num_comb][2] = "hidden_size=" + str(hidden_size) + ", num_layers=" + str(num_layers) \
                    + ", num_epoch=" + str(num_epoch) +   ", learning_rate=" + str(learning_rate)
                print(str(loss_accuracy_comb[num_comb][0]) + ":::" + str(loss_accuracy_comb[num_comb][1]) + ":::" + loss_accuracy_comb[num_comb][2])
                num_comb += 1



[Epoch 1]	Train Loss: 		0.316
[Epoch 2]	Train Loss: 		0.263
[Epoch 3]	Train Loss: 		0.251
[Epoch 4]	Train Loss: 		0.240
[Epoch 5]	Train Loss: 		0.244

Evaluating ... 



Overall Accuracy: 	93.633% 	loss: 0.303
0.3025352493149497:::93.63311767578125:::hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.01


[Epoch 1]	Train Loss: 		0.240
[Epoch 2]	Train Loss: 		0.146
[Epoch 3]	Train Loss: 		0.109
[Epoch 4]	Train Loss: 		0.081
[Epoch 5]	Train Loss: 		0.057

Evaluating ... 



Overall Accuracy: 	94.820% 	loss: 0.223
0.22294541540849336:::94.82037353515625:::hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001


[Epoch 1]	Train Loss: 		0.312
[Epoch 2]	Train Loss: 		0.260
[Epoch 3]	Train Loss: 		0.254
[Epoch 4]	Train Loss: 		0.250
[Epoch 5]	Train Loss: 		0.241
[Epoch 6]	Train Loss: 		0.242
[Epoch 7]	Train Loss: 		0.236
[Epoch 8]	Train Loss: 		0.224
[Epoch 9]	Train Loss: 		0.225
[Epoch 10]	Train Loss: 		0.219

Evaluating ... 


...


Overall Accuracy: 	93.664% 	loss: 0.254
0.25393407787697747:::93.66387939453125:::hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.001


[Epoch 1]	Train Loss: 		0.412
[Epoch 2]	Train Loss: 		0.406
[Epoch 3]	Train Loss: 		0.404
[Epoch 4]	Train Loss: 		0.404
[Epoch 5]	Train Loss: 		0.404
[Epoch 6]	Train Loss: 		0.404
[Epoch 7]	Train Loss: 		0.404
[Epoch 8]	Train Loss: 		0.403
[Epoch 9]	Train Loss: 		0.403
[Epoch 10]	Train Loss: 		0.403

Evaluating ... 



Overall Accuracy: 	93.061% 	loss: 0.471
0.47064464540775136:::93.06101989746094:::hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.01


[Epoch 1]	Train Loss: 		0.371
[Epoch 2]	Train Loss: 		0.305
[Epoch 3]	Train Loss: 		0.227
[Epoch 4]	Train Loss: 		0.203
[Epoch 5]	Train Loss: 		0.195
[Epoch 6]	Train Loss: 		0.179
[Epoch 7]	Train Loss: 		0.165
[Epoch 8]	Train Loss: 		0.153
[Epoch 9]	Train Loss: 		0.143
[Epoch 10]	Train Loss: 		0.138

Evaluating ... 



Overall Accuracy: 	93.602% 	loss: 0.279
0.2789363348061509:::93.60236358642578:::hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.001


In [38]:
with open('val_acc.data', 'wb') as use_val_data_save:
    # store the data as binary data stream
    pickle.dump(loss_accuracy_comb, use_val_data_save)

In [39]:
loss_accuracy_comb

[[0.3025352493149497,
  93.63311767578125,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.01'],
 [0.22294541540849336,
  94.82037353515625,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001'],
 [0.29496811341043055,
  93.86688232421875,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.01'],
 [0.3077647853455875,
  94.21752166748047,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001'],
 [0.463743531071801,
  93.06101989746094,
  'hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.01'],
 [0.2689638561230238,
  93.01181030273438,
  'hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.001'],
 [0.45990698074073283,
  93.06101989746094,
  'hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.01'],
 [0.2798780367828317,
  93.38090515136719,
  'hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.001'],
 [0.37133191239830676,
  91.81840515136719,
  'hidden_size=128, num_layers=2, num_epoch=5, learnin

In [41]:
sorted(loss_accuracy_comb, key = lambda x: x[1], reverse = True)

[[0.22294541540849336,
  94.82037353515625,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001'],
 [0.3180385585215745,
  94.72810363769531,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001'],
 [0.25085918785235634,
  94.62351989746094,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001'],
 [0.3077647853455875,
  94.21752166748047,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001'],
 [0.29496811341043055,
  93.86688232421875,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.01'],
 [0.25393407787697747,
  93.66387939453125,
  'hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.001'],
 [0.3025352493149497,
  93.63311767578125,
  'hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.01'],
 [0.2789363348061509,
  93.60236358642578,
  'hidden_size=128, num_layers=5, num_epoch=5, learning_rate=0.001'],
 [0.2798780367828317,
  93.38090515136719,
  'hidden_size=128, num_layers=5, num_epoch=5, lear

# 📈 Evaluate valid and test data


We achieve the best validation accuracy of 94.82% using hidden_size=128, num_layers=2, num_epoch=5, learning_rate=0.001 . This will be the set up we use for reporting our results.

## Helper functions

In [44]:
import sys
from collections import defaultdict

def split_tag(chunk_tag):
    """
    split chunk tag into IOBES prefix and chunk_type
    e.g. 
    B-PER -> (B, PER)
    O -> (O, None)
    """
    if chunk_tag == 0:
        return ('O', None)
    return list(labels.keys())[chunk_tag].split('-', maxsplit=1)

def is_chunk_end(prev_tag, tag):
    """
    check if the previous chunk ended between the previous and current word
    e.g. 
    (B-PER, I-PER) -> False
    (B-LOC, O)  -> True
    Note: in case of contradicting tags, e.g. (B-PER, I-LOC)
    this is considered as (B-PER, B-LOC)
    """
    prefix1, chunk_type1 = split_tag(prev_tag)
    prefix2, chunk_type2 = split_tag(tag)

    if prefix1 == 'O':
        return False
    if prefix2 == 'O':
        return prefix1 != 'O'

    if chunk_type1 != chunk_type2:
        return True

    return prefix2 in ['B', 'S'] or prefix1 in ['E', 'S']

def is_chunk_start(prev_tag, tag):
    """
    check if a new chunk started between the previous and current word
    """
    prefix1, chunk_type1 = split_tag(prev_tag)
    prefix2, chunk_type2 = split_tag(tag)

    if prefix2 == 'O':
        return False
    if prefix1 == 'O':
        return prefix2 != 'O'

    if chunk_type1 != chunk_type2:
        return True

    return prefix2 in ['B', 'S'] or prefix1 in ['E', 'S']


def calc_metrics(tp, p, t, percent=True):
    """
    compute overall precision, recall and FB1 (default values are 0.0)
    if percent is True, return 100 * original decimal value
    """
    precision = tp / p if p else 0
    recall = tp / t if t else 0
    fb1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    if percent:
        return 100 * precision, 100 * recall, 100 * fb1
    else:
        return precision, recall, fb1


def count_chunks(true_seqs, pred_seqs):
    """
    true_seqs: a list of true tags
    pred_seqs: a list of predicted tags
    return: 
    correct_chunks: a dict (counter), 
                    key = chunk types, 
                    value = number of correctly identified chunks per type
    true_chunks:    a dict, number of true chunks per type
    pred_chunks:    a dict, number of identified chunks per type
    correct_counts, true_counts, pred_counts: similar to above, but for tags
    """
    correct_chunks = defaultdict(int)
    true_chunks = defaultdict(int)
    pred_chunks = defaultdict(int)

    correct_counts = defaultdict(int)
    true_counts = defaultdict(int)
    pred_counts = defaultdict(int)

    prev_true_tag, prev_pred_tag = 0, 0
    correct_chunk = None

    for true_tag, pred_tag in zip(true_seqs, pred_seqs):
        if true_tag == pred_tag:
            correct_counts[true_tag] += 1
        true_counts[true_tag] += 1
        pred_counts[pred_tag] += 1

        _, true_type = split_tag(true_tag)
        _, pred_type = split_tag(pred_tag)

        if correct_chunk is not None:
            true_end = is_chunk_end(prev_true_tag, true_tag)
            pred_end = is_chunk_end(prev_pred_tag, pred_tag)

            if pred_end and true_end:
                correct_chunks[correct_chunk] += 1
                correct_chunk = None
            elif pred_end != true_end or true_type != pred_type:
                correct_chunk = None

        true_start = is_chunk_start(prev_true_tag, true_tag)
        pred_start = is_chunk_start(prev_pred_tag, pred_tag)

        if true_start and pred_start and true_type == pred_type:
            correct_chunk = true_type
        if true_start:
            true_chunks[true_type] += 1
        if pred_start:
            pred_chunks[pred_type] += 1

        prev_true_tag, prev_pred_tag = true_tag, pred_tag
    if correct_chunk is not None:
        correct_chunks[correct_chunk] += 1

    return (correct_chunks, true_chunks, pred_chunks, 
        correct_counts, true_counts, pred_counts)

def get_result(correct_chunks, true_chunks, pred_chunks,
    correct_counts, true_counts, pred_counts, verbose=True):
    """
    if verbose, print overall performance, as well as preformance per chunk type;
    otherwise, simply return overall prec, rec, f1 scores
    """
    # sum counts
    sum_correct_chunks = sum(correct_chunks.values())
    sum_true_chunks = sum(true_chunks.values())
    sum_pred_chunks = sum(pred_chunks.values())

    sum_correct_counts = sum(correct_counts.values())
    sum_true_counts = sum(true_counts.values())
    O_correct_counts = sum(v for k, v in correct_counts.items() if k == 0)
    O_true_counts = sum(v for k, v in true_counts.items() if k == 0)
    O_pred_counts = sum(v for k, v in pred_counts.items() if k == 0)

    chunk_types = sorted(list(set(list(true_chunks) + list(pred_chunks))))

    # compute overall precision, recall and FB1 (default values are 0.0)
    prec, rec, f1 = calc_metrics(sum_correct_chunks, sum_pred_chunks, sum_true_chunks)
    res = (prec, rec, f1)
    if not verbose:
        return res

    print("processed %i tokens with %i phrases; " % (sum_true_counts, sum_true_chunks), end='')
    print("found: %i phrases; correct: %i.\n" % (sum_pred_chunks, sum_correct_chunks), end='')
    print()
    print("%i Entity Types:" % (len(chunk_types)))
    print("accuracy: %6.2f%%; " % (100*sum_correct_counts/sum_true_counts), end='')
    print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" % (prec, rec, f1))

    for t in chunk_types:
        prec, rec, f1 = calc_metrics(correct_chunks[t], pred_chunks[t], true_chunks[t])
        print("%17s: " %t , end='')
        print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" %
                    (prec, rec, f1), end='')
        print("  %d" % pred_chunks[t])

    print()
    print("No Types: ")
    print("accuracy: %6.2f%%; " % (100*O_correct_counts/O_true_counts), end='')
    prec, rec, f1 = calc_metrics(O_correct_counts, O_pred_counts, O_true_counts)
    print("precision: %6.2f%%; recall: %6.2f%%; FB1: %6.2f" % (prec, rec, f1), end='')
    print("  %d" % O_pred_counts)
    return res

def ConLLEval(true_seqs, pred_seqs, verbose=True):
    (correct_chunks, true_chunks, pred_chunks,
        correct_counts, true_counts, pred_counts) = count_chunks(true_seqs, pred_seqs)
    result = get_result(correct_chunks, true_chunks, pred_chunks,
        correct_counts, true_counts, pred_counts, verbose=verbose)
    return result

## Run

In [42]:
# Based on perplexity on validation data, we choose these values for hyperparamters
hidden_dim=128
num_layers=2
epochs=5
learning_rate=0.001

# train model
classifier = RNN(hidden_dim, num_layers).to(device)
train(classifier, device, hidden_dim, num_layers, epochs, learning_rate)

[Epoch 1]	Train Loss: 		0.224
[Epoch 2]	Train Loss: 		0.142
[Epoch 3]	Train Loss: 		0.104
[Epoch 4]	Train Loss: 		0.073
[Epoch 5]	Train Loss: 		0.054



In [45]:
print('[Validation Data]')
preds, truth, loss, accuracy = evaluate(classifier, device, valid_embeddings, valid_tags)
print("Loss:", loss)
print("Accuracy:", loss)

print()
ConLLEval(truth, preds)

[Validation Data]
Evaluating ... 



Overall Accuracy: 	94.660% 	loss: 0.239
Loss: 0.2394547949938103
Accuracy: 0.2394547949938103

processed 16256 tokens with 661 phrases; found: 688 phrases; correct: 282.

10 Entity Types:
accuracy:  94.66%; precision:  40.99%; recall:  42.66%; FB1:  41.81
          company: precision:  17.07%; recall:  53.85%; FB1:  25.93  123
         facility: precision:  16.36%; recall:  23.68%; FB1:  19.35  55
          geo-loc: precision:  63.20%; recall:  68.10%; FB1:  65.56  125
            movie: precision:  25.00%; recall:   6.67%; FB1:  10.53  4
      musicartist: precision:   9.52%; recall:   9.76%; FB1:   9.64  42
            other: precision:  10.94%; recall:   5.30%; FB1:   7.14  64
           person: precision:  61.24%; recall:  74.85%; FB1:  67.37  209
          product: precision:   5.00%; recall:   2.70%; FB1:   3.51  20
       sportsteam: precision:  74.42%; recall:  45.71%; FB1:  56.64  43
           tvshow: precision:   0.00%; recall:   0.00%; FB1:   0.00  3

No Types: 
accuracy: 

(40.98837209302326, 42.662632375189105, 41.808747220163085)

In [46]:
print('[Test Data]')
preds, truth, loss, accuracy = evaluate(classifier, device, test_embeddings, test_tags)
print("Loss:", loss)
print("Accuracy:", loss)

print()
ConLLEval(truth, preds)

[Test Data]
Evaluating ... 



Overall Accuracy: 	92.332% 	loss: 0.383
Loss: 0.38252955272909056
Accuracy: 0.38252955272909056

processed 61880 tokens with 3473 phrases; found: 3305 phrases; correct: 1249.

10 Entity Types:
accuracy:  92.33%; precision:  37.79%; recall:  35.96%; FB1:  36.85
          company: precision:  34.34%; recall:  47.67%; FB1:  39.92  862
         facility: precision:  20.09%; recall:  17.79%; FB1:  18.87  224
          geo-loc: precision:  61.42%; recall:  55.78%; FB1:  58.47  801
            movie: precision:   0.00%; recall:   0.00%; FB1:   0.00  10
      musicartist: precision:  11.86%; recall:   7.33%; FB1:   9.06  118
            other: precision:  10.25%; recall:   4.97%; FB1:   6.69  283
           person: precision:  38.37%; recall:  68.46%; FB1:  49.18  860
          product: precision:  10.26%; recall:   1.63%; FB1:   2.81  39
       sportsteam: precision:  40.43%; recall:  25.85%; FB1:  31.54  94
           tvshow: precision:   7.14%; recall:   3.03%; FB1:   4.26  14

No Types: 


(37.791225416036305, 35.96314425568672, 36.85452935969312)